# PaLM API: Text Quickstart with Python

- [Google LLM Guide](https://developers.generativeai.google/guide/concepts)
- https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb

In [1]:
import pprint
import google.generativeai as palm

In [2]:
import api_key_store as aks

In [3]:
s = aks.ApiKeyStore()

In [4]:
palm.configure(api_key=s.get_api_key("GOOGLE"))

## Text generation

Use the palm.list_models function to find available models:

In [5]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]

In [6]:
models

[Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='Text Bison',
       description='Model targeted for text generation.',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       top_p=0.95,
       top_k=40)]

In [7]:
model = models[0].name
print(model)

models/text-bison-001


In [8]:
prompt = """
You are an expert at solving word problems.

Solve the following problem:

I have three houses, each with three cats.
each cat owns 4 mittens, and a hat. Each mitten was
knit from 7m of yarn, each hat from 4m.
How much yarn was needed to make all the items?

Think about it step by step, and show your work in separated paragraphs.
"""

In [9]:
completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # The maximum length of the response
    max_output_tokens=800,
)

In [10]:
print(completion.result)

**Step 1: Determine the total number of cats.**

There are three houses, each with three cats, for a total of 3 * 3 = 9 cats.

**Step 2: Determine the total number of mittens.**

Each cat has 4 mittens, for a total of 9 cats * 4 mittens / cat = 36 mittens.

**Step 3: Determine the total number of hats.**

Each cat has 1 hat, for a total of 9 cats * 1 hat / cat = 9 hats.

**Step 4: Determine the total amount of yarn needed for the mittens.**

Each mitten requires 7 meters of yarn, for a total of 36 mittens * 7 meters / mitten = 252 meters of yarn.

**Step 5: Determine the total amount of yarn needed for the hats.**

Each hat requires 4 meters of yarn, for a total of 9 hats * 4 meters / hat = 36 meters of yarn.

**Step 6: Add the amounts of yarn needed for the mittens and hats to find the total amount of yarn needed.**

The total amount of yarn needed is 252 meters + 36 meters = 288 meters.

**The final answer: 288 meters.**


In [11]:
calc_prompt = f"""
Please solve the following problem:

{prompt}

----------------

Important: Use the calculator for each step.
Don't do the arithmetic in your head. 

To use the calculator wrap an equation in  tags like this: 

 3 cats * 2 hats/cat  = 6

----------------

Solve the problem in step by step, and show your work in logical paragraphs.
"""

Not working with while-loop

In [12]:
completion = palm.generate_text(
    model=model,
    prompt=calc_prompt,
    temperature=0,
    # The maximum length of the response
    max_output_tokens=800,
)

In [13]:
print(completion.result)

**Step 1:**

There are 3 houses with 3 cats each, for a total of 3 * 3 = 9 cats.

**Step 2:**

Each cat has 4 mittens, for a total of 9 cats * 4 mittens / cat = 36 mittens.

**Step 3:**

Each hat takes 4m of yarn, for a total of 9 cats * 1 hat / cat = 9 hats.

**Step 4:**

Each mitten takes 7m of yarn, for a total of 36 mittens * 7m / mitten = 252m of yarn.

**Step 5:**

Each hat takes 4m of yarn, for a total of 9 hats * 4m / hat = 36m of yarn.

**Step 6:**

The total amount of yarn needed is 252m + 36m = 288m.

**Thus, the answer is 288.**


## Candidates

In [15]:
completion = palm.generate_text(
    model=model,
    prompt=prompt,
    # The number of candidates to return
    candidate_count=8,
    # Set the temperature to 1.0 for more variety of responses.
    temperature=1.0,
    max_output_tokens=800,
)

In [16]:
import pprint
pprint.pprint(completion.candidates)

[{'output': '1. Chain-of-thought:\n'
            '\n'
            'First, find the total number of cats: 3 houses * 3 cats / house = '
            '9 cats. Then, multiply that number by the number of mittens per '
            'cat to find the total number of mittens: 9 cats * 4 mittens / cat '
            '= 36 mittens. Then, multiply that number by the length of yarn '
            'per mitten to find the total length of yarn used for mittens: 36 '
            'mittens * 7m / mitten = 252m. Then, multiply the number of cats '
            'by the number of hats per cat to find the total number of hats: 9 '
            'cats * 1 hat / cat = 9 hats. Then, multiply that number by the '
            'length of yarn per hat to find the total length of yarn used for '
            'hats: 9 hats * 4m / hat = 36m. Then, add the total lengths of '
            'yarn used for mittens and hats to find the total length of yarn '
            'used: 252m + 36m = 288m.\n'
            '\n'
            '2.

So, since you know the answer to this problem, it's easy to check the solve rate:

In [17]:
import numpy as np
np.mean(['288' in c['output'] for c in completion.candidates])

0.625